# OSMUtils Examples
When developing, to test newly saved code:

- restart kernel
- after each save in the command line use: !pip install -e . to install osmUtils from local directory
- if this imports with no issues, the code it good!
- then re-import osmUtils

-examples here: https://wiki.openstreetmap.org/wiki/Map_features#Leisure

# 1. Install and import osmUtils

In [19]:
!pip install -e .

from IPython.display import clear_output
clear_output()

import osmUtils as osmu
from osmUtils import utils_geo, utils_osm, utils_map

print(f'osmUtils ver. {osmu.__version__} ready!')

osmUtils ver. 0.0.1 ready!


# 2. Find a location to retrieve data from

In [20]:

import LMIPy as lmi #Docs https://lmipy.readthedocs.io/en/latest/quickstart.html#From-Political-Boundaries

params={
    'iso': 'USA',
    'adm1': 33,
    'adm2': 32
}

ny_geom = lmi.Geometry(parameters=params)
ny_geom.map()

In [21]:
# get the geometry
geom = ny_geom.shape()

# 3. Generate a gdf collection form your input geometry

As part of the osmUTils, we have incorporated a CollectionOsm main class. This collection class will produce a gdf tile manifest at an especific zoom level to keep track of the OSM retrieving process. The idea behind this class would be helping the user keeping track of consecutive retreaving for a a big area of interest, where you want to divide the location in different chunks or tiles. 

You can also create a gdf manifest for your input geometry (withouth dividing it in tiles). This will also help you with the tracking of the retrieving process. 

As result, you will have a geoDataFrame and a map visualization with the geometry where you are retreiving the data from:

In [4]:
col = osmu.CollectionOsm(geometry=geom[0], zoom=5, crs=None, geom_tiles=False)
col

/home/jovyan/work/test_dev/osmUtils/osmUtils/utils_geo.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manifest['exclude'], manifest['exported'], manifest['uploaded'] = [0, 0, 0]


In [5]:
# access to the geodataframe manifest for keeping track of the retrieving process:
manifest = col.manifest
manifest

,id,geometry,exclude,exported,uploaded
300,0.0,"POLYGON ((-73.98656 40.79379, -73.96997 40.816...",0,0,0


# 4. Retrieve osm data from Overpass API

The osmu.OsmDownload class interfaces directly with overpass API to retrieve nodes a ways for a given geometry. In this function you can retrieve directly the body in json format for a given geometry and for a given filter but also, you would be able to incorporate the metadata to this request. 

The filters to you request can be some predefined ones but you can add any customised one. The filters that we have already customised are:

- 'all_roads': selects major and minor roads
- main_roads: selects primary roads
- 'river': the linear flow of a river, in flow direction
- 'water_features': any inland body of water, from natural such as a lake or pond to artificial like a moat or canal
- coastline: The mean high (springs) line between the sea and land (with the water on the right side of the way)
- forest: Managed forest or woodland plantation (Other languages). Some use this to map an area of trees rather than the use of the land. See Forest.
- building: The building tags are intended for the physical description of a building
- parks: A park, usually urban (municipal)


In [6]:
#select geometry to retrieve data
geometry = manifest.iloc[0].geometry

## 4.1 Retrieve data:

In [7]:
#retrieve data without metadata
osm_Data_roads = osmu.OsmDownload(
    geometry,
    osm_type='main_roads'
)


Fetching OSM
Geometry coordines converted into string
40.793789 -73.986557 40.816410 -73.969971 40.878582 -73.935776 40.873779 -73.912819 40.869671 -73.910477 40.865139 -73.911194 40.848961 -73.923706 40.839039 -73.928787 40.820950 -73.932831 40.807751 -73.929466 40.800861 -73.921158 40.796062 -73.926941 40.798611 -73.928055 40.794693 -73.929733 40.790791 -73.935997 40.785816 -73.938477 40.782265 -73.944374 40.775070 -73.942085 40.755520 -73.961861 40.743332 -73.971664 40.733124 -73.974426 40.726395 -73.971703 40.711445 -73.976868 40.709469 -73.982132 40.709049 -73.994286 40.706284 -74.000000 40.707291 -74.000275 40.704948 -74.001114 40.702339 -74.005180 40.700851 -74.015480 40.705956 -74.019958 40.715279 -74.017502 40.714443 -74.024261 40.759029 -74.013000 40.793789 -73.986557
[out:json][timeout:180];(way["highway"="primary"](poly:"40.793789 -73.986557 40.816410 -73.969971 40.878582 -73.935776 40.873779 -73.912819 40.869671 -73.910477 40.865139 -73.911194 40.848961 -73.923706 40.8390

In [8]:
#retrieve data with metadata
osm_Data_roads_meta = osmu.OsmDownload(
    geometry,
    osm_type='main_roads',
    includes_metadata=True
)


Fetching OSM
Geometry coordines converted into string
40.793789 -73.986557 40.816410 -73.969971 40.878582 -73.935776 40.873779 -73.912819 40.869671 -73.910477 40.865139 -73.911194 40.848961 -73.923706 40.839039 -73.928787 40.820950 -73.932831 40.807751 -73.929466 40.800861 -73.921158 40.796062 -73.926941 40.798611 -73.928055 40.794693 -73.929733 40.790791 -73.935997 40.785816 -73.938477 40.782265 -73.944374 40.775070 -73.942085 40.755520 -73.961861 40.743332 -73.971664 40.733124 -73.974426 40.726395 -73.971703 40.711445 -73.976868 40.709469 -73.982132 40.709049 -73.994286 40.706284 -74.000000 40.707291 -74.000275 40.704948 -74.001114 40.702339 -74.005180 40.700851 -74.015480 40.705956 -74.019958 40.715279 -74.017502 40.714443 -74.024261 40.759029 -74.013000 40.793789 -73.986557
[out:json][timeout:180];(way["highway"="primary"](poly:"40.793789 -73.986557 40.816410 -73.969971 40.878582 -73.935776 40.873779 -73.912819 40.869671 -73.910477 40.865139 -73.911194 40.848961 -73.923706 40.8390

## 4.2 Generate gdf:

In [9]:
# get the gdf from your response retrieved
roads_gdf = osm_Data_roads.get_osm_gdf()
roads_gdf.head()

,geometry,id
0,"LINESTRING (-73.93422 40.80683, -73.93451 40.8...",5669595
1,"LINESTRING (-73.93942 40.84173, -73.93959 40.8...",5669799
2,"LINESTRING (-73.99217 40.71968, -73.99209 40.7...",5669857
3,"LINESTRING (-73.99409 40.72034, -73.99419 40.7...",5669892
4,"LINESTRING (-74.01284 40.70244, -74.01288 40.7...",5669933


In [10]:
roads_gdf_meta = osm_Data_roads_meta.get_osm_gdf()
roads_gdf_meta

,geometry,id,type,timestamp,version,changeset,user,uid,nodes,tags
0,"LINESTRING (-73.93422 40.80683, -73.93451 40.8...",5669595,way,2020-12-29T20:08:01Z,20,96622711,Fluffy89502,10165657,"[371337140, 3786741766, 3786741765, 3786741764...","{'destination:street': '2nd Avenue', 'hgv': 'l..."
1,"LINESTRING (-73.93942 40.84173, -73.93959 40.8...",5669799,way,2021-01-15T23:57:23Z,34,97586941,Mateusz Konieczny,1722488,"[2499757467, 7132814245]","{'cycleway:right': 'lane', 'hgv': 'local', 'hi..."
2,"LINESTRING (-73.99217 40.71968, -73.99209 40.7...",5669857,way,2021-01-10T01:27:47Z,29,97233821,kguenther24,11697944,"[486867432, 8231896494, 8307642254, 486867405]","{'cycleway:left': 'lane', 'hgv': 'designated',..."
3,"LINESTRING (-73.99409 40.72034, -73.99419 40.7...",5669892,way,2021-01-10T01:27:47Z,17,97233821,kguenther24,11697944,"[42429752, 5296795596, 8231909737, 42430030, 8...","{'hgv': 'designated', 'highway': 'primary', 'm..."
4,"LINESTRING (-74.01284 40.70244, -74.01288 40.7...",5669933,way,2020-07-05T15:51:59Z,14,87558925,grid1811plan,9272243,"[42426747, 2034011049, 1772058064, 2034011044,...","{'1': '2', 'hgv': 'local', 'highway': 'primary..."
...,...,...,...,...,...,...,...,...,...,...
1330,"LINESTRING (-73.97928 40.78184, -73.97882 40.7...",895398384,way,2021-01-15T03:44:43Z,1,97525378,Sparks,80285,"[42438862, 42428063, 6929267036]","{'hgv': 'local', 'highway': 'primary', 'lanes'..."
1331,"LINESTRING (-73.97863 40.78274, -73.97833 40.7...",895398385,way,2021-01-15T03:44:43Z,1,97525378,Sparks,80285,"[6929267036, 42442463]","{'hgv': 'local', 'highway': 'primary', 'lanes'..."
1332,"LINESTRING (-73.98626 40.72229, -73.98639 40.7...",903593425,way,2021-02-03T22:19:21Z,1,98668660,TheBestIdea,298513,"[42455039, 4598752856, 5783020909, 4478001045]","{'cycleway:right': 'lane', 'hgv': 'destination..."
1333,"LINESTRING (-73.99024 40.72350, -73.99035 40.7...",903593426,way,2021-02-03T22:19:21Z,1,98668660,TheBestIdea,298513,"[42451665, 5783028393, 3927743131, 4776041130]","{'cycleway:right': 'lane', 'hgv': 'designated'..."


## 4.3 Visualize gdf:

In [11]:
#visualize the data retrieved
osmu.OsmVisualize(roads_gdf, color='#a7a9ab')

In [12]:
#visualize the data with metadata
osmu.OsmVisualize(roads_gdf_meta, color='#a7a9ab')

## 4.4 Save gdf

Saves the retrieved data in a local data file (path = './data'). The save_gdf_to_file class has been built on top of geopandas.

In [16]:
import geopandas as gpd

In [13]:
osm_Data_roads.save_gdf_to_file(
    filename='osm_data',
    driver='ESRI Shapefile',
)

In [15]:
!ls data

osm_data.cpg  osm_data.dbf  osm_data.shp  osm_data.shx


In [18]:
gpd.read_file('./data/osm_data.shp')

,id,geometry
0,5669595,"LINESTRING (-73.93422 40.80683, -73.93451 40.8..."
1,5669799,"LINESTRING (-73.93942 40.84173, -73.93959 40.8..."
2,5669857,"LINESTRING (-73.99217 40.71968, -73.99209 40.7..."
3,5669892,"LINESTRING (-73.99409 40.72034, -73.99419 40.7..."
4,5669933,"LINESTRING (-74.01284 40.70244, -74.01288 40.7..."
...,...,...
1330,895398384,"LINESTRING (-73.97928 40.78184, -73.97882 40.7..."
1331,895398385,"LINESTRING (-73.97863 40.78274, -73.97833 40.7..."
1332,903593425,"LINESTRING (-73.98626 40.72229, -73.98639 40.7..."
1333,903593426,"LINESTRING (-73.99024 40.72350, -73.99035 40.7..."
